In [17]:
import pandas as pd
from datetime import datetime
import math
import warnings
import numpy as np
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')

In [28]:
df = pd.read_csv('CRSP_Updated_Clean_2019.csv')
df['date'] = pd.to_datetime(df['date'])

In [29]:
df.head()

,date,stock_id,ticker,shrout_adj_factor,price,return_incl_divs,shrout,return_ex_divs,mktrf,smb,hml,rf,mom,split_dummy,div_init_dummy
0,1987-01-30,10000,OMFGA,NaN,0.406250,-0.212121,3893.0,-0.212121,0.1247,-0.0181,-0.0318,0.0042,0.0210,0,0
1,1987-02-27,10000,OMFGA,NaN,0.406250,0.000000,3893.0,0.000000,0.0439,0.0349,-0.0599,0.0043,-0.0217,0,0
2,1987-03-31,10000,OMFGA,NaN,0.250000,-0.384615,3893.0,-0.384615,0.0164,0.0037,0.0166,0.0047,0.0160,0,0
3,1987-04-30,10000,OMFGA,NaN,0.234375,-0.062500,3893.0,-0.062500,-0.0211,-0.0169,-0.0033,0.0044,0.0026,0,0
4,1987-05-29,10000,OMFGA,NaN,0.218750,-0.066667,3893.0,-0.066667,0.0011,-0.0053,0.0013,0.0038,-0.0068,0,0


In [30]:
df['month'] = pd.DatetimeIndex(df['date']).month
df['year'] = pd.DatetimeIndex(df['date']).year
df.head()

,date,stock_id,ticker,shrout_adj_factor,price,return_incl_divs,shrout,return_ex_divs,mktrf,smb,hml,rf,mom,split_dummy,div_init_dummy,month,year
0,1987-01-30,10000,OMFGA,NaN,0.406250,-0.212121,3893.0,-0.212121,0.1247,-0.0181,-0.0318,0.0042,0.0210,0,0,1,1987
1,1987-02-27,10000,OMFGA,NaN,0.406250,0.000000,3893.0,0.000000,0.0439,0.0349,-0.0599,0.0043,-0.0217,0,0,2,1987
2,1987-03-31,10000,OMFGA,NaN,0.250000,-0.384615,3893.0,-0.384615,0.0164,0.0037,0.0166,0.0047,0.0160,0,0,3,1987
3,1987-04-30,10000,OMFGA,NaN,0.234375,-0.062500,3893.0,-0.062500,-0.0211,-0.0169,-0.0033,0.0044,0.0026,0,0,4,1987
4,1987-05-29,10000,OMFGA,NaN,0.218750,-0.066667,3893.0,-0.066667,0.0011,-0.0053,0.0013,0.0038,-0.0068,0,0,5,1987


In [33]:
p_div = {}
p_split = {}
p_split_div = {}

split_div = df[['stock_id','year','month','split_dummy','div_init_dummy']]

for i in df['year'].unique():
    p_div[i+1] = {}
    p_split[i+1] = {}
    p_split_div[i+1] = {}
    for j in df['month'].unique():
        temp = split_div[ ((split_div['year']==i) & (split_div['month']>=j)) | ((split_div['year']==i+1) & (split_div['month']<j))]
        split_div_sum = temp[['stock_id','split_dummy','div_init_dummy']].groupby(['stock_id']).sum()
        p_div[i+1][j] = split_div_sum[split_div_sum['div_init_dummy']>0].index
        p_split[i+1][j] = split_div_sum[split_div_sum['split_dummy']>0].index
        p_split_div[i+1][j] = list(set(p_div[i+1][j].append(p_split[i+1][j])))

In [124]:
data2 = df[['stock_id', 'ticker', 'price', 'return_incl_divs', 'shrout', 'mktrf', 'rf', 'year', 'month']]
returns = []
risk_free = []
mktrf = []
for i in df['year'].unique():
    for j in df['month'].unique():
        if i < 2018:
            temp = data2[(data2['stock_id'].isin(p_div[i+1][j]))&(data2['year']==i+1)&(data2['month']==j)]
            temp['MC']=temp['price']*temp['shrout']
            temp['weight']=temp['MC']/temp['MC'].sum()
            temp['Return']=temp['weight']*temp['return_incl_divs']
            returns.append(temp['Return'].sum())
            risk_free.append(temp['rf'].mean())
            mktrf.append(temp['mktrf'].mean())
        else:
            pass

In [121]:
returns[:5]

[0.04761945323154981,
 0.07590353555016462,
 0.014100925744639132,
 0.023015247346270657,
 -0.018399700974654484]

In [120]:
risk_free[:5]

[0.002899999999999986,
 0.004599999999999978,
 0.004399999999999972,
 0.00459999999999998,
 0.005100000000000054]

In [127]:
from operator import sub
excess_returns = map(sub, returns, risk_free)
excess_returns[:5]

[0.04471945323154983,
 0.07130353555016464,
 0.00970092574463916,
 0.018415247346270678,
 -0.023499700974654536]

In [128]:
mktrf[:5]

[0.04209999999999996,
 0.0474999999999997,
 -0.022700000000000217,
 0.0056,
 -0.0028999999999999894]

In [132]:
model_df = pd.DataFrame(
    {'excess_returns': excess_returns,
     'mktrf': mktrf,
    })
model_df[:5]

,excess_returns,mktrf
0,0.044719,0.0421
1,0.071304,0.0475
2,0.009701,-0.0227
3,0.018415,0.0056
4,-0.023500,-0.0029


Avg Return 

In [100]:
avg_return = sum(returns)/(len(returns))
avg_return

0.016362396953879123

Standard Deviation

In [101]:
std = np.std(returns)
std

0.053486585893367286

Sharpe Ratio

In [104]:
sharpe_ratio = avg_return/std
sharpe_ratio

0.30591589798795094

In [133]:
import statsmodels.api as sm
from patsy import dmatrices
import numpy as np

y, X = dmatrices('excess_returns ~ mktrf', data=model_df, return_type='dataframe')
model = sm.OLS(y, X)       # Set up the model
result = model.fit()       # Fit model (find the intercept and slopes)
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:         excess_returns   R-squared:                       0.625
Model:                            OLS   Adj. R-squared:                  0.624
Method:                 Least Squares   F-statistic:                     617.8
Date:                Sat, 05 Oct 2019   Prob (F-statistic):           6.68e-81
Time:                        18:30:01   Log-Likelihood:                 744.91
No. Observations:                 372   AIC:                            -1486.
Df Residuals:                     370   BIC:                            -1478.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0072      0.002      4.197      0.000       0.004       0.011
mktrf          1.0136      0.041     24.855      0.000       0.933       1.094
==============================================================================
Omnibus:                      302.050   Durbin-Watson:                   1.822
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             9641.454
Skew:                           3.100   Prob(JB):                         0.00
Kurtosis:                      27.157   Cond. No.                         24.0
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Sharpe Ratio first half vs. second half 

In [135]:
np.mean(returns[:int(len(returns)/2)])

0.02034609108047913

In [140]:
ret_half1 = returns[:int(len(returns)/2)]
std_half1 = np.std(ret_half1)
SR_half1 = np.mean(ret_half1) / std_half1

ret_half2 = returns[int(len(returns)/2):]
std_half2 = np.std(ret_half2)
SR_half2 = np.mean(ret_half2) / std_half2

print"Sharpe Ratio of the first period:",SR_half1
print"Sharpe Ratio of the second period:",SR_half2

Sharpe Ratio of the first period: 0.3380102431098262
Sharpe Ratio of the second period: 0.27229907681038146


Split Portfolio Value Weighted

In [141]:
returns_split = []
risk_free_split = []
mktrf_split = []
for i in df['year'].unique():
    for j in df['month'].unique():
        if i < 2018:
            temp1 = data2[(data2['stock_id'].isin(p_split[i+1][j]))&(data2['year']==i+1)&(data2['month']==j)]
            temp1['MC']=temp1['price']*temp1['shrout']
            temp1['weight']=temp1['MC']/temp1['MC'].sum()
            temp1['Return']=temp1['weight']*temp1['return_incl_divs']
            returns_split.append(temp1['Return'].sum())
            risk_free_split.append(temp1['rf'].mean())
            mktrf_split.append(temp1['mktrf'].mean())
        else:
            pass

In [145]:
from operator import sub
excess_returns_split = map(sub, returns_split, risk_free_split)
excess_returns_split[:5]

[0.04956276393984159,
 0.04541328239410075,
 -0.03273188068498918,
 0.011262196757020244,
 0.010593530727921846]

In [146]:
model_df_split = pd.DataFrame(
    {'excess_returns': excess_returns_split,
     'mktrf': mktrf_split,
    })
model_df_split[:5]

,excess_returns,mktrf
0,0.049563,0.0421
1,0.045413,0.0475
2,-0.032732,-0.0227
3,0.011262,0.0056
4,0.010594,-0.0029


In [150]:
avg_return_split = sum(returns_split)/(len(returns_split))
avg_return_split

0.017182223161835958

In [156]:
std_split = np.std(returns_split)
std_split

0.051456064764227365

In [172]:
sharpe_ratio_split = avg_return_split/std_split
sharpe_ratio_split

0.3339202723831529

In [159]:
import statsmodels.api as sm
from patsy import dmatrices
import numpy as np

y, X = dmatrices('excess_returns ~ mktrf', data=model_df_split, return_type='dataframe')
model = sm.OLS(y, X)       # Set up the model
result = model.fit()       # Fit model (find the intercept and slopes)
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:         excess_returns   R-squared:                       0.735
Model:                            OLS   Adj. R-squared:                  0.734
Method:                 Least Squares   F-statistic:                     1025.
Date:                Sat, 05 Oct 2019   Prob (F-statistic):          1.12e-108
Time:                        18:50:28   Log-Likelihood:                 823.22
No. Observations:                 372   AIC:                            -1642.
Df Residuals:                     370   BIC:                            -1635.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0077      0.001      5.560      0.000       0.005       0.010
mktrf          1.0578      0.033     32.017      0.000       0.993       1.123
==============================================================================
Omnibus:                       40.553   Durbin-Watson:                   1.984
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              147.978
Skew:                           0.393   Prob(JB):                     7.36e-33
Kurtosis:                       5.988   Cond. No.                         24.0
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [161]:
np.mean(returns_split[:int(len(returns_split)/2)])

0.020223839510069987

In [162]:
ret_half1_split = returns_split[:int(len(returns)/2)]
std_half1_split = np.std(ret_half1_split)
SR_half1_split = np.mean(ret_half1_split) / std_half1_split

ret_half2_split = returns_split[int(len(returns_split)/2):]
std_half2_split = np.std(ret_half2_split)
SR_half2_split = np.mean(ret_half2_split) / std_half2_split

print"Sharpe Ratio of the first period:",SR_half1_split
print"Sharpe Ratio of the second period:",SR_half2_split

Sharpe Ratio of the first period: 0.36536689382713533
Sharpe Ratio of the second period: 0.30058577088802463


Dividend or Split Portfolio Value Weighted

In [143]:
returns_split_div = []
risk_free_split_div = []
mktrf_split_div = []
for i in df['year'].unique():
    for j in df['month'].unique():
        if i < 2018:
            temp2 = data2[(data2['stock_id'].isin(p_split_div[i+1][j]))&(data2['year']==i+1)&(data2['month']==j)]
            temp2['MC']=temp2['price']*temp2['shrout']
            temp2['weight']=temp2['MC']/temp2['MC'].sum()
            temp2['Return']=temp2['weight']*temp2['return_incl_divs']
            returns_split_div.append(temp2['Return'].sum())
            risk_free_split_div.append(temp2['rf'].mean())
            mktrf_split_div.append(temp2['mktrf'].mean())
        else:
            pass

In [178]:
temp

,stock_id,ticker,price,return_incl_divs,shrout,mktrf,rf,year,month,MC,weight,Return
103334,11018,FBP,8.600000,-0.049724,217241.0,-0.0955,0.0019,2018,12,1.868273e+06,0.003721,-1.850467e-04
196579,11990,KKR,19.629999,-0.143543,536337.0,-0.0955,0.0019,2018,12,1.052829e+07,0.020972,-3.010348e-03
223341,12362,STND,29.879999,0.010484,4810.0,-0.0955,0.0019,2018,12,1.437228e+05,0.000286,3.001322e-06
226436,12477,WD,43.250000,-0.084268,31239.0,-0.0955,0.0019,2018,12,1.351087e+06,0.002691,-2.267909e-04
231271,12622,HCA,124.449997,-0.135704,344160.0,-0.0955,0.0019,2018,12,4.283071e+07,0.085316,-1.157777e-02
239283,13046,UBNT,99.410004,-0.087814,70835.0,-0.0955,0.0019,2018,12,7.041708e+06,0.014027,-1.231744e-03
250993,13534,HTBI,26.180000,0.007311,18659.0,-0.0955,0.0019,2018,12,4.884926e+05,0.000973,7.113509e-06
253347,13625,SSTK,36.009998,-0.057577,35037.0,-0.0955,0.0019,2018,12,1.261682e+06,0.002513,-1.447015e-04
253725,13641,FANG,92.699997,-0.160174,101258.0,-0.0955,0.0019,2018,12,9.386616e+06,0.018698,-2.994872e-03
260948,13908,PFSI,21.260000,0.036568,77491.0,-0.0955,0.0019,2018,12,1.647459e+06,0.003282,1.200017e-04


In [163]:
from operator import sub
excess_returns_split_div = map(sub, returns_split_div, risk_free_split_div)
excess_returns_split_div[:5]

[0.04917903431368595,
 0.04786679245926139,
 -0.02753879597772921,
 0.011812774289766632,
 0.005805007230283298]

In [164]:
model_df_split_div = pd.DataFrame(
    {'excess_returns': excess_returns_split_div,
     'mktrf': mktrf_split_div,
    })
model_df_split_div[:5]

,excess_returns,mktrf
0,0.049179,0.0421
1,0.047867,0.0475
2,-0.027539,-0.0227
3,0.011813,0.0056
4,0.005805,-0.0029


In [166]:
avg_return_split_div = sum(returns_split_div)/(len(returns_split_div))
avg_return_split_div

0.016447397484885527

In [168]:
std_split_div = np.std(returns_split_div)
std_split_div

0.04909672360981725

In [169]:
sharpe_ratio_div = avg_return_split_div/std_split_div
sharpe_ratio_div

0.33499989969996186

In [170]:
import statsmodels.api as sm
from patsy import dmatrices
import numpy as np

y, X = dmatrices('excess_returns ~ mktrf', data=model_df_split_div, return_type='dataframe')
model = sm.OLS(y, X)       # Set up the model
result = model.fit()       # Fit model (find the intercept and slopes)
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:         excess_returns   R-squared:                       0.830
Model:                            OLS   Adj. R-squared:                  0.830
Method:                 Least Squares   F-statistic:                     1807.
Date:                Sat, 05 Oct 2019   Prob (F-statistic):          1.82e-144
Time:                        18:56:03   Log-Likelihood:                 923.72
No. Observations:                 372   AIC:                            -1843.
Df Residuals:                     370   BIC:                            -1836.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0069      0.001      6.506      0.000       0.005       0.009
mktrf          1.0720      0.025     42.512      0.000       1.022       1.122
==============================================================================
Omnibus:                       34.323   Durbin-Watson:                   1.845
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               79.693
Skew:                           0.469   Prob(JB):                     4.95e-18
Kurtosis:                       5.064   Cond. No.                         24.0
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [171]:
ret_half1_split_div = returns_split_div[:int(len(returns)/2)]
std_half1_split_div = np.std(ret_half1_split_div)
SR_half1_split_div = np.mean(ret_half1_split_div) / std_half1_split_div

ret_half2_split_div = returns_split_div[int(len(returns_split_div)/2):]
std_half2_split_div = np.std(ret_half2_split_div)
SR_half2_split_div = np.mean(ret_half2_split_div) / std_half2_split_div

print"Sharpe Ratio of the first period:",SR_half1_split_div
print"Sharpe Ratio of the second period:",SR_half2_split_div

Sharpe Ratio of the first period: 0.36979908042470844
Sharpe Ratio of the second period: 0.29815799727262876
